## Infos ( à supprimer ensuite)

## Variables à ajouter issues des recherches biblio ( à supprimer ensuite)

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 
EXTRA POUR AVOIR 20 : Attribuer un poid aux médailles -> pour potentiellement savoir le nb d'or , argent , bronze

Attention au COVID 19 , influence public , web scrapping sur 30 ans de tous les pays qui ont organisé les jeux 
Attention aux pays pas autorisés à concourir 
Pays en guerre qui envoient moins d'athlètes 
Ukraine 
Palestine 
Iles caiman -> doivent être rattachées à un pays 

JO hiver 
JO été Corrélation 
Apparition des femmes dans les JO ( échantillonnage ) 

Culture du sport : Comment la créer ? 
Budget alloué aux sports (investissement , infrastructures ... ) 
Nb de professionnels 
Nb de licencés 
Nb de sports olympiques disponibles à l'école 

Variable : Pays Communiste ou pas 

Croissance pays économique 

Athlète à partir de 25 ans il n'a plus le niveau : on commence à partir de 2000 , JO Sydney 

Indice dvp des pays à recevoir 

Variable pays organisateur 

Sport qui ont des gens avec un âge bien définie (exemple skate Japonais c'est que des jeunes <= 18 ans)

Dataset coupes mondes de discipline  
    
Variable Note selon l'âge 

Athlète qui sont comptés comme participant à deux pays 

Variable : religion majoritaire pays 

Espérance de vie par pays 
Variable pays dvp ou pas 


Recupérer données JO 2018 et 2021

Variable Poids par Médailles : échelloner les JO de 1 à 10 pour 
Variable Poids par année : 

# Introduction  (Business Understanding) 

## Détermination objectifs métier , analytique du projet 

## Identifier les contraintes , limites et risque du projet

## Chargement des packages si besoin 

In [5]:
%load_ext ipython_memory_usage 

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


In [6]:
 #!pip install ydata_profiling

In [ ]:
#%imu_stop

## Importation des librairies 

In [ ]:
from ydata_profiling import ProfileReport

In [8]:
import pandas as pd
import numpy as np
import ipython_memory_usage.ipython_memory_usage as imu
from describe_csv import *  # charge le fichier describe_csv.py par chunk de 10 000 (10**4)

import math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy as sp
import scipy.stats
from pivottablejs import pivot_ui
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

imu.start_watching_memory() 

In [8] used 12.5 MiB RAM in 60353.76s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 62.6 MiB


# CODE  

## Bases de données disponibles 

In [ ]:
# df_athlète=
#df_region=
#df_dictionnary=

## Optimisation des datasets
   Dans un souci d'optimisation de l'espace mémoire occupé, nous allons faire en sorte d'optimiser les différents dataframe que nous utilisons. 
Pour mener à bien ceci, nous allons des fonctions contenues dans `describe_csv.py`, en les améliorant pour automatiser le processus.

In [ ]:
#Code

In [ ]:
def optimize_data_types(df):
    """
    Optimize data types of columns in a pandas DataFrame.

    Parameters:
        df (pandas.DataFrame): Input DataFrame.

    Returns:
        pandas.DataFrame: DataFrame with optimized data types.
    """
    optimized_df = df.copy()
    
    for col in optimized_df.columns:
        col_dtype = optimized_df[col].dtype
        
        if col_dtype == 'object':
            # Convert object type columns to category if unique values are less than 50% of total rows
            if len(optimized_df[col].unique()) / len(optimized_df[col]) < 0.5:
                optimized_df[col] = optimized_df[col].astype('category')
        elif col_dtype == 'int64':
            # Convert int64 to smaller integer types if possible
            optimized_df[col] = pd.to_numeric(optimized_df[col], downcast='integer', errors='coerce')
        elif col_dtype == 'float64':
            # Convert float64 to smaller floating point types if possible
            optimized_df[col] = pd.to_numeric(optimized_df[col], downcast='float', errors='coerce')
        elif col_dtype == 'bool':
            # Convert boolean columns to integer type
            optimized_df[col] = optimized_df[col].astype('int8')

    return optimized_df

In [ ]:
#df_test['Age'] = pd.to_numeric(df_test['Age'], errors='coerce', downcast='integer')

# Convert 'Height' and 'Weight' columns to int16
#df_test['Height'] = pd.to_numeric(df_test['Height'], errors='coerce', downcast='integer')
#df_test['Weight'] = pd.to_numeric(df_test['Weight'], errors='coerce', downcast='integer')

In [ ]:
#### Code Nathan si tu veux rajouter 

### Modification des noms pays ... (Code Guillaume)

In [ ]:
# Explications texte 

In [ ]:
#df['NOC'] = df['NOC'].replace(['Soviet Union', 'URS'], 'RUS')

#df['Team'] = df['Team'].replace(['Soviet Union', 'URS'], 'Russia')
#df['Team'] = df['Team'].replace(['East Germany'], 'Germany')

In [ ]:
#df_athlete=df_athlete.query(" NOC !='ROC' and NOC != 'UNK' ") 
#df_regions=df_regions.query(" NOC != 'ROT' and NOC != 'UNK' ")


Suppression refugiés , vous avez pas de pays lol ... et UNK c'est UNK ou encore une autre appelation de refugiés jsp plus 

### Modification noms variables 

In [ ]:
#df_regions=df_regions.rename(columns={'region':'Country'}) 
#df_dictionnary=df_dictionnary.rename(columns={'GDP per Capita':'PIB/habitant','Code':'NOC'})


### Chargement de la base de données commune  ( On va fusionner mais faut avoir changé les petits couak des 3 df) 

In [ ]:
# Code 

In [ ]:
# df=pd.merge(pd.merge(df_athlete,df_dictionnary,on='Country'how='left'),df_region,on='Country',how='left')

In [ ]:
### Attribution score et poids médaille 

## Traitement des valeurs manquantes 

Effectivement notre base de données possède de nombreuses valeurs manquantes 

In [ ]:
# df.isna().sum()

In [ ]:
### Première solution : Web Scrapping (code Nathan + FLO)

# Data understanding

## EDA (graph, pivot table ,Matrix confusion...)

In [3]:
#df=pd.read_csv('data/data_FLO_without_NaN.csv',sep=',')